<a href="https://colab.research.google.com/github/Jass1313c/comp215/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime, json, requests
from pprint import pprint

param = { "format": "geojson", "starttime": "2024-11-14","limit": 20000, "endtime": "2024-11-15", "minmagnitude": 3.6,}
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
response = requests.get(url, params=param)
data = json.loads(response.text)

pprint(f"Count: {data['metadata']['count']}")

pprint(data)

pprint(f"Count: {data['metadata']['count']}")

for feature in data['features']:
    print(f"Mag: {feature['properties']['mag']}, Place: {feature['properties']['place']}, Time: {datetime.datetime.fromtimestamp(feature['properties']['time'] / 1000)}")

In [ ]:
import datetime, json, requests
from pprint import pprint


url = "https://data.nasa.gov/resource/gh4g-9sfh.geojson"

response = requests.get(url)
data = json.loads(response.text)

pprint(data)

In [44]:
import datetime, json, requests
from pprint import pprint
from bs4 import BeautifulSoup

url = "https://volcanoes.usgs.gov/vhp/updates.html"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing volcano updates
# The original class name might have changed or be incorrect.
# Inspect the website's HTML to find the correct table and class name.
# Here's an example using a different selector if the table has an id:
table = soup.find('table', {'id': 'some_table_id'})

# Check if the table was found before proceeding
if table:
    # Extract data from each row of the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        volcano_name = columns[0].text.strip()
        update_time = columns[1].text.strip()
        update_type = columns[2].text.strip()
        update_summary = columns[3].text.strip()

        print(f"Volcano: {volcano_name}")
        print(f"Update Time: {update_time}")
        print(f"Update Type: {update_type}")
        print(f"Update Summary: {update_summary}")
        print("-" * 20)
else:
    print("Table not found. Check the website's HTML for the correct table and class name.")

Table not found. Check the website's HTML for the correct table and class name.
